In [1]:

import regex as re
import pandas as pd
import numpy as np
from termcolor import colored
from urllib.request import urlopen
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [9]:
%pwd 
%cd c:\\Users\\Lenovo\\Flask apps\\House-price-prediction\\

c:\Users\Lenovo\Flask apps\House-price-prediction


In [10]:
MAIN_DF_PATH = r'Dataset\calgary_cleaned_rentals_dataset.csv'
df = pd.read_csv(MAIN_DF_PATH)

In [11]:

df.dropna(subset=['price'], inplace=True)
df.reset_index(inplace=True, drop=True)


In [13]:
malformatted_dates_idx = []
for i in range(len(df)):
    try: pd.to_datetime(df.loc[i, 'a'], format='%Y-%m-%d')
    except: malformatted_dates_idx.append(i)


In [15]:
df.loc[[malformatted_dates_idx[1]], :]

,rented,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,sq_feet2,price2,beds2,baths2,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
147,Not-Rented,NaN,3099-01-01,Calgary,Bridgeland,51.05425,-114.050074,/ab/calgary/rentals/apartment/studio/bridgeland/pet-friendly/481774,Apartment,995.0,studio,NaN,1,2,2,NaN,NaN,0,0.0,1,1,1,0,0


In [17]:
len(df[(~df.price2.isna())&(~df.sq_feet2.isna())])

189

In [29]:
def clean_undefinite_dates(df, null_substitute = 'call for availability'):
    for i in range(len(df)):
        try: 
            df.loc[i, 'a'] = pd.to_datetime(df.loc[i, 'a'], format='%Y-%m-%d')
        except: 
            df.loc[i, 'a'] = null_substitute
            df.loc[i, 'availability'] = null_substitute
    df.reset_index(inplace=True)
    return df
    
df = clean_undefinite_dates(df)

In [31]:
df[df.sq_feet.isna()]

,index,rented,availability,a,city,community,latitude,longitude,link,type,price,beds,sq_feet,baths,cats,dogs,sq_feet2,price2,beds2,baths2,utility_heat,utility_electricity,utility_water,utility_cable,utility_internet
0,0,Not-Rented,Immediate,2000-01-01 00:00:00,Calgary,Manchester,51.002873,-114.068828,/ab/calgary/rentals/apartment/1-bedroom/manchester/468894,Apartment,580.0,1,NaN,1,0,0,NaN,NaN,0,0.0,0,0,0,0,0
1,1,Not-Rented,Immediate,2000-01-01 00:00:00,Calgary,Downtown,51.047811,-114.059262,/ab/calgary/rentals/apartment/studio/downtown/472999,Apartment,580.0,studio,NaN,1,0,0,NaN,NaN,0,0.0,1,1,1,0,0
4,4,Not-Rented,Immediate,2000-01-01 00:00:00,Calgary,Downtown,51.047297,-114.082807,/ab/calgary/rentals/apartment/1-bedroom/downtown/477769,Apartment,960.0,1,NaN,1,0,0,NaN,NaN,0,0.0,0,0,0,0,0
6,6,Not-Rented,Immediate,2000-01-01 00:00:00,Calgary,Beltline,51.039233,-114.078444,/ab/calgary/rentals/apartment/1-bedroom/beltline/452842,Apartment,925.0,1,NaN,1,0,0,NaN,NaN,0,0.0,1,0,1,0,0
8,8,Not-Rented,Immediate,2000-01-01 00:00:00,Calgary,Banff Trail,51.067452,-114.115680,/ab/calgary/rentals/apartment/2-bedrooms/banff-trail/pet-friendly/403379,Apartment,940.0,studio,NaN,1,2,2,NaN,NaN,0,0.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2555,Not-Rented,May 15,2022-05-15 00:00:00,Calgary,New Brighton,50.914957,-113.941632,/ab/calgary/rentals/townhouse/3-bedrooms/new-brighton/pet-friendly/410679,Townhouse,1950.0,3,NaN,2.5,2,2,NaN,NaN,0,0.0,0,0,0,0,0
2560,2560,Not-Rented,Jun 01,2022-06-01 00:00:00,Calgary,New Brighton,50.927703,-113.956913,/ab/calgary/rentals/townhouse/3-bedrooms/new-brighton/pet-friendly/484322,Townhouse,1950.0,3,NaN,2.5,2,2,NaN,NaN,0,0.0,1,1,1,0,0
2576,2576,Not-Rented,Jun 01,2022-06-01 00:00:00,Calgary,West Springs,51.062876,-114.211714,/ab/calgary/rentals/townhouse/3-bedrooms/west-springs/484066,Townhouse,2300.0,3+den,NaN,2.5,0,0,NaN,NaN,0,0.0,0,0,0,0,0
2591,2591,Not-Rented,Jul 01,2022-07-01 00:00:00,Calgary,Evanston,51.168168,-114.129226,/ab/calgary/rentals/townhouse/2-bedrooms/evanston/128500,Townhouse,1700.0,2,NaN,2.5,0,0,NaN,NaN,0,0.0,0,0,0,0,0


In [27]:
df[df.availability != 'Immediate'].availability.unique()
#df.loc[:, 'availability_month'] 

array(['May 01', 'Apr 15', 'Apr 30', 'call for availability', 'May 15',
       'Apr 07', 'Apr 17', 'Apr 20', 'Apr 21', 'Apr 28', 'May 06',
       'May 07', 'May 09', 'May 10', 'May 13', 'May 20', 'Jun 01',
       'Jun 15', 'Jul 01', 'Jul 15', 'Aug 01', 'Sep 01', 'Apr 13',
       'Apr 14', 'Jun 30', 'Nov 01', 'Apr 11', 'Apr 09', 'Apr 10',
       'Apr 12', 'Apr 16', 'May 26', 'Apr 08', 'May 05', 'May 31',
       'Jul 08', 'Oct 01', 'Apr 18', 'May 02', 'May 16', 'Jun 05',
       'Sep 24', 'Jun 20', 'Jun 22', 'Jun 02', 'Apr 25', 'May 03',
       'May 18', 'May 21', 'May 28', 'Jun 07', 'Jun 16'], dtype=object)